In [1]:
import os
import sys
from pathlib import Path

mod_path = os.path.join(Path.cwd().parent.parent)
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy

from sklearn.model_selection import StratifiedKFold

from src.data import *
from src.features.utils import *
from src.model.tree_based import ModelXgBoost

In [3]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train.csv')
test_set = pd.read_csv('../../data/processed/test.csv')

In [4]:
train

,Unnamed: 0,ID,Deposit,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,...,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid,b1,b2,b3,b4,b5
0,0,ID_MR53LEX,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,517.26,350.0,3600.0,55.0,652.49,770.0,280.0,1320.0,1200.0,660.0
1,1,ID_3D7NQUH,2500,0.0,DAILY,55,3,Male,26.0,South Rift,...,737.87,655.0,2940.0,380.0,452.03,770.0,655.0,660.0,660.0,605.0
2,2,ID_0IWQNPI,2400,0.0,DAILY,50,3,Male,21.0,Mount Kenya Region,...,1126.67,980.0,2850.0,200.0,1005.32,250.0,200.0,610.0,1350.0,1500.0
3,3,ID_IY8SYB9,2000,0.0,DAILY,40,7,Female,26.0,Mount Kenya Region,...,1126.00,1140.0,2200.0,380.0,511.30,380.0,600.0,1100.0,1300.0,780.0
4,4,ID_9XHL7VZ,2000,0.0,DAILY,40,7,Male,27.0,North Rift,...,353.59,190.0,2640.0,40.0,559.24,520.0,80.0,40.0,40.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,28002,ID_CDZ24L9,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,1753.21,1635.0,5235.0,800.0,1069.37,1750.0,1600.0,1500.0,800.0,1670.0
28003,28003,ID_0XINELS,2400,0.0,DAILY,50,3,Female,NaN,Nairobi Region,...,1166.25,1025.0,2980.0,450.0,786.66,550.0,1200.0,1250.0,850.0,1050.0
28004,28004,ID_PAU9JJU,2000,0.0,DAILY,40,3,Male,29.0,South Rift,...,952.22,1020.0,2580.0,200.0,695.70,1020.0,1110.0,1080.0,660.0,200.0
28005,28005,ID_K866QHS,2000,0.0,DAILY,40,7,Female,57.0,Mount Kenya Region,...,1241.25,1180.0,2000.0,970.0,323.04,970.0,1240.0,1200.0,1000.0,1120.0


In [5]:
train.drop(columns = 'Unnamed: 0', inplace=True)
test_set.drop(columns = 'Unnamed: 0', inplace=True)
test_set.columns

Index(['ID', 'Deposit', 'AccessoryRate', 'rateTypeEntity', 'RatePerUnit',
       'DaysOnDeposit', 'MainApplicantGender', 'Age', 'Region', 'Occupation',
       'Term', 'TotalContractValue', 'ExpectedTermDate', 'FirstPaymentDate',
       'LastPaymentDate', 'SplitPaymentsHistory', 'nb_payments', 'amount_paid',
       'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid', 'max_amt_paid',
       'min_amt_paid', 'stddev_amt_paid', 'b1', 'b2', 'b3', 'b4', 'b5'],
      dtype='object')

In [6]:
train.shape

(28007, 35)

In [7]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(train['Region'].isna().sum() / train.shape[0] * 100)
train.dropna(subset=['Region'], how='all', inplace=True)

5.162994965544328


In [ ]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(test_set['Region'].isna().sum() / test_set.shape[0] * 100)
# train.dropna(subset=['Region'], how='all', inplace=True)

# When attempting drop=first in OHE, the reverse transform throws an issue as it reads the NaN values as a separate
# category. So converting NaNs into strings
train['Region'] = train['Region'].fillna('Null')
test_set['Region'] = test_set['Region'].fillna('Null')

In [ ]:
train['SplitPaymentsHistory'][0]

In [ ]:
train

## Approach 2

s = pd.DataFrame(np.arange(0, len(train)), columns=['m1'])
df = train[['b1', 'b2', 'b3', 'b4', 'b5']]
df.drop(columns=['b5'], inplace=True)
df.rename(columns={'b1': 'b2', 'b2':'b3', 'b3': 'b4', 'b4': 'b5'}, inplace=True)
df.insert(loc=0, column='b1', value=s.values)
df

t = pd.DataFrame([[1]], columns=['a'])
q = pd.DataFrame([[1]], columns=['b'])
r = pd.DataFrame([['aum']], columns=['name'])

k = pd.concat([t, q])
kk = pd.merge(k, r, how='left', left_index=True, right_index=True)
kk

In [8]:
def slide_variable_window(
    predictor_array: pd.DataFrame,
    var_to_add: pd.DataFrame
) -> pd.DataFrame:
    predictor_array.drop(columns=['b5'], inplace=True)  # We drop the first payment
    predictor_array.rename(columns={'b1': 'b2', 'b2':'b3', 'b3': 'b4', 'b4': 'b5'}, inplace=True)
    predictor_array.insert(loc=0, column='b1', value=var_to_add.values)  # And add the new variable (mn)
    
    return predictor_array

In [9]:
# split_payment_history_df = train[["ID", "SplitPaymentsHistory"]]
# id_arr = train[["ID"]]

target = train[['m1', 'm2', 'm3', 'm4', 'm5', 'm6']]
train_arr = train.drop(columns=['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 
                                'SplitPaymentsHistory',
                                'ExpectedTermDate', 
                                'FirstPaymentDate',
                                'LastPaymentDate'])
test_set.drop(columns=['SplitPaymentsHistory',
                       'ExpectedTermDate', 
                       'FirstPaymentDate',
                       'LastPaymentDate'], inplace=True)

In [10]:
def create_data_with_sliding_approach(data_without_target: pd.DataFrame,
                                      target_data: pd.DataFrame):
    frame = pd.DataFrame(None)
    new_df = deepcopy(data_without_target)
    target_df = pd.DataFrame(None)
    target_features = target_data.columns.tolist()
    for itr, col in enumerate(target_features):
        if itr == 0:
            target_df = pd.concat([target_df, target_data[[col]]])
            frame = pd.concat([frame, data_without_target])
        else:
            filter_df = new_df[['b1', 'b2', 'b3', 'b4', 'b5']]  # Intermediate df
            new_df.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
            concatinating_df = slide_variable_window(predictor_array=filter_df, 
                                                     var_to_add=target_data[[target_features[itr-1]]])
            new_df = pd.concat([new_df, concatinating_df], axis=1)  # We add the newly created columns
            target_df = pd.concat([target_df, target_data[[col]]])
            frame = pd.concat([frame, new_df])
#             print(new_df.shape)

    target_df = pd.DataFrame(target_df.sum(axis=1).astype(int), columns=['target'])
#     print(frame.shape)  # Should be 6 * original data's no. of rows
    
    frame.reset_index(drop=True, inplace=True)
    target_df.reset_index(drop=True, inplace=True)
    
    return frame, target_df

In [11]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

seed = 10
X_train, X_test, y_train, y_test = train_test_split(train_arr, target, test_size=0.45, random_state=seed)

In [12]:
X_train

,ID,Deposit,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Occupation,...,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid,b1,b2,b3,b4,b5
6770,ID_IC22NG8,2000,0.0,DAILY,40,7,Male,37.0,Nairobi Region,Driver/Motorbike Rider,...,602.17,480.0,2200.0,80.0,444.97,1080.0,920.0,480.0,820.0,480.0
14803,ID_NRJ4U6A,2999,0.0,DAILY,55,3,Male,NaN,Western,Business,...,333.12,137.5,3055.0,55.0,729.86,165.0,275.0,275.0,110.0,55.0
1708,ID_4P4FX8T,2500,0.0,DAILY,55,3,Male,NaN,Nairobi Region,Business,...,1571.67,1500.0,3800.0,885.0,740.21,1700.0,1100.0,1350.0,885.0,1500.0
12676,ID_VT3WHGH,2500,0.0,DAILY,55,3,Female,NaN,North Rift,Farmer,...,1352.11,1265.0,2720.0,385.0,516.72,990.0,1870.0,1430.0,385.0,2310.0
3648,ID_LUE2J1I,2000,0.0,DAILY,40,7,Female,48.0,Nyanza,Farmer,...,477.86,400.0,2340.0,100.0,406.41,250.0,300.0,560.0,500.0,590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9833,ID_54ZHDD0,2000,0.0,DAILY,40,3,Male,29.0,Western,Teacher,...,1265.00,1120.0,2100.0,940.0,429.97,990.0,940.0,1320.0,1140.0,1100.0
7635,ID_P5WECCB,2000,0.0,DAILY,40,7,Female,52.0,Coast Region,Business,...,1487.14,1200.0,3480.0,310.0,1005.93,2000.0,1200.0,1200.0,1080.0,1140.0
18686,ID_V3OH30J,2999,0.0,DAILY,55,3,Male,24.0,South Rift,Business,...,1434.58,495.0,10770.0,55.0,2535.36,165.0,495.0,440.0,58.0,385.0
7638,ID_IOWZ2BT,2999,0.0,DAILY,55,3,Female,30.0,Nairobi Region,Farmer,...,1588.67,1410.0,3610.0,825.0,682.25,1860.0,930.0,1485.0,2035.0,1250.0


### Model train on initial hp :: Approach 2

def approach_two_model(x_train, y_train, x_test):
    model = ModelXgBoost(train_array=x_train, train_target=y_train)
    model.train_model()  # Default h.params (Checkout the code)
    predict = model.trained_model.predict(x_test)
    
    return model, predict

In [13]:
def encode_and_drop(full_array, data_type, tr_encoder=None):
#     print(tr_encoder)
    categorical_array = full_array[full_array.select_dtypes(exclude=['number']).columns]
    numerical_array = full_array.drop(columns=full_array.select_dtypes(exclude=['number']).columns)
    print(numerical_array.shape)
    encoded_array, encoder = one_hot_encoding(
        categorical_frame=categorical_array, 
        type_of_data=data_type,
        fitted_encoder=tr_encoder,
        conv=True,
        drop=None,
        handle_unknown="ignore"
    )
#     print(encoded_array.shape)
    final_array = pd.concat([numerical_array.reset_index(drop=True), 
                             encoded_array.reset_index(drop=True)], axis=1)
    final_array.index = numerical_array.index
#     print(final_array.shape)
    return numerical_array, encoded_array, encoder

In [14]:
# Preparing the TRAIN data for approach two and fitting the model
train_data, target_frame = create_data_with_sliding_approach(data_without_target=X_train, 
                                                             target_data=y_train)
id_array = train_data[["ID"]]
train_data.drop(columns=["ID"], inplace=True)
og_frame, encoded_train, encoder_model = encode_and_drop(train_data, "train", None)
encoded_train = pd.concat([encoded_train, train_data[['b1', 'b2', 'b3', 'b4', 'b5']]], axis=1)

model_two_obj = ModelXgBoost(train_array=encoded_train, 
                             train_target=target_frame)
model_two_obj.train_model()  # Default h.params (Checkout the code)
model_two = model_two_obj.trained_model

/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


(87648, 20)
{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'gamma': None, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 50, 'reg_lambda': 0, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': 0, 'use_label_encoder': False}


### Prediction using Model :: Approach 2

In [15]:
test_id_array = X_test[["ID"]]
X_test.drop(columns=["ID"], inplace=True)

# Encoding and re-attaching using train encoding model
og_frame_test, encoded_test, encoder_model = encode_and_drop(X_test, "test", encoder_model)
encoded_test = pd.concat([encoded_test, X_test[['b1', 'b2', 'b3', 'b4', 'b5']].reset_index(drop=True)], axis=1)

predict_dict = dict()

for col_no, predict_col in enumerate(y_test.columns):
    predict_dict[f"m{col_no+1}_pred"] = model_two.predict(encoded_test)
    int_df = encoded_test[['b1', 'b2', 'b3', 'b4', 'b5']]
    encoded_test.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
    concatinating_df = slide_variable_window(predictor_array=int_df, 
                                             var_to_add=pd.DataFrame(predict_dict[f"m{col_no+1}_pred"]))
    encoded_test = pd.concat([encoded_test, concatinating_df], axis=1)  # We add the newly created columns
    
predict_dict

(11953, 20)


/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

{'m1_pred': array([ 298.98547, 1370.1808 ,  336.08096, ...,  229.55864, 1385.3893 ,
        1319.7733 ], dtype=float32),
 'm2_pred': array([ 365.25754, 1408.1921 ,  404.01724, ...,  223.63448, 1303.6301 ,
        1298.9755 ], dtype=float32),
 'm3_pred': array([ 321.3584 , 1430.3431 ,  447.55276, ...,  223.63448, 1314.6316 ,
        1222.1951 ], dtype=float32),
 'm4_pred': array([ 379.00632, 1279.6415 ,  432.97665, ...,  209.94781, 1338.343  ,
        1241.9469 ], dtype=float32),
 'm5_pred': array([ 359.7216 , 1228.2202 ,  387.7058 , ...,  263.27692, 1250.7705 ,
        1142.567  ], dtype=float32),
 'm6_pred': array([ 414.9437 , 1168.9843 ,  365.17502, ...,  247.06192, 1127.6016 ,
        1129.6669 ], dtype=float32)}

### Calculation of RMSE

In [16]:
pred_frame = pd.DataFrame(predict_dict)
pred_frame.index = X_test.index

In [17]:
full_test_array = pd.concat([X_test, y_test, pred_frame], axis=1)

In [18]:
full_test_array = pd.merge(full_test_array, test_id_array, how='left', left_index=True, right_index=True)

In [19]:
sub_file = SubmissionFile(
    validation_data=full_test_array,
    type_of_data='validation'
).execute()

In [20]:
sub_file.shape

(71718, 3)

In [21]:
sub_file['SquaredError'] = np.square(sub_file['Target'] - sub_file['Prediction'])

In [22]:
rmse = np.sqrt(np.sum(sub_file['SquaredError'])/sub_file.shape[0])
print('Final RMSE --> ', rmse)

Final RMSE -->  836.3801053455517


### Preparing Submission

In [23]:
# Preparing the Entire TRAIN data for approach two and fitting the model
train_data, target_frame = create_data_with_sliding_approach(data_without_target=train_arr, 
                                                             target_data=target)
id_array = train_data[["ID"]]
train_data.drop(columns=["ID"], inplace=True)
og_frame, encoded_train, encoder_model = encode_and_drop(train_data, "train", None)
encoded_train = pd.concat([encoded_train, train_data[['b1', 'b2', 'b3', 'b4', 'b5']]], axis=1)

model_two_obj = ModelXgBoost(train_array=encoded_train, 
                             train_target=target_frame)
model_two_obj.train_model()  # Default h.params (Checkout the code)
model_two = model_two_obj.trained_model

(159366, 20)
{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'gamma': None, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 50, 'reg_lambda': 0, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': 0, 'use_label_encoder': False}


In [24]:
test_id = test_set[["ID"]]
test_set.drop(columns=["ID"], inplace=True)

# Encoding and re-attaching using train encoding model
og_frame_test, encoded_test, encoder_model = encode_and_drop(test_set, "test", encoder_model)
encoded_test = pd.concat([encoded_test, test_set[['b1', 'b2', 'b3', 'b4', 'b5']].reset_index(drop=True)], axis=1)

predict_dict = dict()

for col_no, predict_col in enumerate(['m1', 'm2', 'm3', 'm4', 'm5', 'm6']):
    predict_dict[f"m{col_no+1}_pred"] = model_two.predict(encoded_test)
    int_df = encoded_test[['b1', 'b2', 'b3', 'b4', 'b5']]
    encoded_test.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
    concatinating_df = slide_variable_window(predictor_array=int_df, 
                                             var_to_add=pd.DataFrame(predict_dict[f"m{col_no+1}_pred"]))
    encoded_test = pd.concat([encoded_test, concatinating_df], axis=1)  # We add the newly created columns
    
predict_dict

(9336, 20)


/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


{'m1_pred': array([ 661.51556,  994.4691 ,  451.49948, ...,  510.37296,  424.38858,
        1120.2953 ], dtype=float32),
 'm2_pred': array([ 731.32043, 1016.4169 ,  550.83746, ...,  487.77353,  470.10187,
        1034.0331 ], dtype=float32),
 'm3_pred': array([715.85675, 965.6536 , 558.9649 , ..., 459.66986, 387.92603,
        983.3424 ], dtype=float32),
 'm4_pred': array([705.01   , 916.36053, 637.6042 , ..., 447.2155 , 388.20517,
        926.0551 ], dtype=float32),
 'm5_pred': array([659.5263 , 897.4925 , 619.4237 , ..., 475.4942 , 421.4239 ,
        892.35583], dtype=float32),
 'm6_pred': array([651.8438 , 842.8392 , 642.2441 , ..., 451.69653, 451.63226,
        842.13855], dtype=float32)}

In [25]:
pred_frame_test = pd.DataFrame(predict_dict)
pred_frame_test.index = test_set.index

In [26]:
full_test_array_test = pd.concat([test_set, pred_frame_test], axis=1)
full_test_array_test = pd.merge(full_test_array_test, test_id, how='left', left_index=True, right_index=True)

In [27]:
sub_file = SubmissionFile(
    validation_data=full_test_array_test,
    type_of_data='test'
).execute()
sub_file.reset_index(drop=True, inplace=True)

In [28]:
sub_file.to_csv('../../submissions/submission_approach_2_dropping_null_regions.csv', )

In [29]:
sub_file

,ID,Target
0,ID_000RHRU x m1,190.413239
1,ID_000RHRU x m2,206.070969
2,ID_000RHRU x m3,201.010620
3,ID_000RHRU x m4,272.594757
4,ID_000RHRU x m5,290.183105
...,...,...
56011,ID_ZZOKWZJ x m2,902.918884
56012,ID_ZZOKWZJ x m3,870.154114
56013,ID_ZZOKWZJ x m4,821.411133
56014,ID_ZZOKWZJ x m5,825.815063
